## Initial Setup & Imports

In [1]:
import numpy
print numpy.__version__

1.11.1


In [2]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed

## Parse Date Function

In [16]:
def parse_date(date):
    return datetime.datetime.strptime(date, "%m/%d/%Y")

## RowParser

This class parses a single-row of the CBOE raw future file

First couple of lines: 

```
CFE data is compiled for the convenience of site visitors and is furnished without responsibility for accuracy and is accepted by the site visitor on the condition that transmission or omissions shall not be made the basis for any claim demand or cause for action. The information and data was obtained from sources believed to be reliable but accuracy is not guaranteed. Your use of CFE data is subject to the Terms and Conditions of CBOE's Websites.
Trade Date,Futures,Open,High,Low,Close,Settle,Change,Total Volume,EFP,Open Interest
04/20/2015,F (Jan 16),0.00,18.95,0.00,0.00,18.9750,18.98,0,0,0
```

See ```parseBar``` for the mapping of csv columns to the ```bar.BasicBar``` of PyAlgoTrade.

In [21]:
from pyalgotrade.barfeed import csvfeed

class RowParser(csvfeed.RowParser):
    def __init__(self, dailyBarTime, frequency, timezone=None, sanitize=False):
        self.__dailyBarTime = dailyBarTime
        self.__frequency = frequency
        self.__timezone = timezone
        self.__sanitize = sanitize

    def __parseDate(self, dateString):
        ret = parse_date(dateString)
        # Time on Yahoo! Finance CSV files is empty. If told to set one, do it.
        if self.__dailyBarTime is not None:
            ret = datetime.datetime.combine(ret, self.__dailyBarTime)
        # Localize the datetime if a timezone was given.
        if self.__timezone:
            ret = dt.localize(ret, self.__timezone)
        return ret

    def getFieldNames(self):
        # It is expected for the first row to have the field names.
        return None

    def getDelimiter(self):
        return ","

    def parseBar(self, csvRowDict):
        dateTime = self.__parseDate(csvRowDict["Trade Date"])
        close = float(csvRowDict["Close"])
        open_ = float(csvRowDict["Open"])
        high = float(csvRowDict["High"])
        low = float(csvRowDict["Low"])
        volume = float(csvRowDict["Total Volume"])
        adjClose = float(csvRowDict["Close"])

        if self.__sanitize:
            open_, high, low, close = common.sanitize_ohlc(open_, high, low, close)
        
        if high < low: #fix for bad bars where the open interest is 0, and you have weird conditions
            return None
        
        return bar.BasicBar(dateTime, open_, high, low, close, volume, adjClose, self.__frequency)


## Fetch the CSV File and Sanitizing It

In [18]:
import urllib2
response = urllib2.urlopen('http://cfe.cboe.com/Publish/ScheduledTask/MktData/datahouse/CFE_F16_VX.csv')
html = response.read()
html_processed = "\n".join(html.split("\n")[1:])
print html_processed

filename = "sanitized_vix.csv"
target = open(filename, 'w')
target.write(html_processed)
target.close()

Trade Date,Futures,Open,High,Low,Close,Settle,Change,Total Volume,EFP,Open Interest
04/20/2015,F (Jan 16),0.00,18.95,0.00,0.00,18.9750,18.98,0,0,0
04/21/2015,F (Jan 16),0.00,19.05,19.65,0.00,19.3500,0.38,0,0,0
04/22/2015,F (Jan 16),0.00,19.20,19.25,0.00,19.1500,-0.20,0,0,0
04/23/2015,F (Jan 16),0.00,19.10,19.25,0.00,19.1750,0.02,0,0,0
04/24/2015,F (Jan 16),19.15,19.15,19.15,19.15,19.1750,0.00,2,0,2
04/27/2015,F (Jan 16),19.25,19.30,19.15,19.25,19.3500,0.18,1,0,3
04/28/2015,F (Jan 16),19.45,19.45,19.10,19.10,19.1250,-0.22,3,0,4
04/29/2015,F (Jan 16),19.05,19.25,19.05,19.25,19.1750,0.05,3,0,6
04/30/2015,F (Jan 16),19.27,19.45,19.23,19.30,19.3750,0.20,9,0,10
05/01/2015,F (Jan 16),19.20,19.25,19.05,19.05,19.1000,-0.28,4,0,10
05/04/2015,F (Jan 16),19.15,19.20,19.07,19.20,19.1750,0.08,6,0,14
05/05/2015,F (Jan 16),19.30,19.31,19.25,19.25,19.2750,0.10,9,0,22
05/06/2015,F (Jan 16),19.30,19.52,19.25,19.35,19.3250,0.05,126,0,253
05/07/2015,F (Jan 16),19.55,19.56,19.25,19.30,19.3000,-0.02,81,0,250

## Test RowParser

In [30]:
from pyalgotrade.utils import csvutils
import datetime
import pprint

#ripped from PyAlgoTrade's internal function:
#def addBarsFromCSV(self, instrument, path, rowParser):

path = filename
rowParser =  RowParser(datetime.time(0, 0, 0), bar.Frequency.DAY, None, False)

loadedBars = []
reader = csvutils.FastDictReader(open(path, "r"), fieldnames=rowParser.getFieldNames(), delimiter=rowParser.getDelimiter())
for row in reader:
    bar_ = rowParser.parseBar(row)
    
    if bar_ is not None:
        loadedBars.append(bar_)


print "Length of Loaded Bars:"
print len(loadedBars)

print "First Bar:"
print loadedBars[0].__getstate__()

Length of Loaded Bars:
188
First Bar:
(datetime.datetime(2015, 4, 20, 0, 0), 0.0, 0.0, 18.95, 0.0, 0.0, 0.0, 86400, False)


## Create a PyAlgoFeed Feed and Strategy

In [31]:
from pyalgotrade.barfeed import common
from pyalgotrade.utils import dt
from pyalgotrade import bar
from pyalgotrade import dataseries

class Feed(csvfeed.BarFeed):
    """A :class:`pyalgotrade.barfeed.csvfeed.BarFeed` that loads bars from CSV files downloaded from Yahoo! Finance.

    :param frequency: The frequency of the bars. Only **pyalgotrade.bar.Frequency.DAY** or **pyalgotrade.bar.Frequency.WEEK**
        are supported.
    :param timezone: The default timezone to use to localize bars. Check :mod:`pyalgotrade.marketsession`.
    :type timezone: A pytz timezone.
    :param maxLen: The maximum number of values that the :class:`pyalgotrade.dataseries.bards.BarDataSeries` will hold.
        Once a bounded length is full, when new items are added, a corresponding number of items are discarded from the opposite end.
    :type maxLen: int.

    .. note::
        Yahoo! Finance csv files lack timezone information.
        When working with multiple instruments:

            * If all the instruments loaded are in the same timezone, then the timezone parameter may not be specified.
            * If any of the instruments loaded are in different timezones, then the timezone parameter must be set.
    """

    def __init__(self, frequency=bar.Frequency.DAY, timezone=None, maxLen=dataseries.DEFAULT_MAX_LEN):
        if isinstance(timezone, int):
            raise Exception("timezone as an int parameter is not supported anymore. Please use a pytz timezone instead.")

        if frequency not in [bar.Frequency.DAY, bar.Frequency.WEEK]:
            raise Exception("Invalid frequency.")

        csvfeed.BarFeed.__init__(self, frequency, maxLen)
        self.__timezone = timezone
        self.__sanitizeBars = False

    def sanitizeBars(self, sanitize):
        self.__sanitizeBars = sanitize

    def barsHaveAdjClose(self):
        return True

    def addBarsFromCSV(self, instrument, path, timezone=None):
        """Loads bars for a given instrument from a CSV formatted file.
        The instrument gets registered in the bar feed.

        :param instrument: Instrument identifier.
        :type instrument: string.
        :param path: The path to the CSV file.
        :type path: string.
        :param timezone: The timezone to use to localize bars. Check :mod:`pyalgotrade.marketsession`.
        :type timezone: A pytz timezone.
        """

        if isinstance(timezone, int):
            raise Exception("timezone as an int parameter is not supported anymore. Please use a pytz timezone instead.")

        if timezone is None:
            timezone = self.__timezone

        rowParser = RowParser(self.getDailyBarTime(), self.getFrequency(), timezone, self.__sanitizeBars)
        csvfeed.BarFeed.addBarsFromCSV(self, instrument, path, rowParser)

In [33]:
from pyalgotrade import strategy 

class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        strategy.BacktestingStrategy.__init__(self, feed)
        self.__instrument = instrument

    def onBars(self, bars):
        bar = bars[self.__instrument]
        self.info(bar.getClose())

In [34]:
# Load the yahoo feed from the CSV file
feed = Feed()
feed.addBarsFromCSV("VX_F", "sanitized_vix.csv")

In [35]:
# Evaluate the strategy with the feed's bars.
myStrategy = MyStrategy(feed, "VX_F")
myStrategy.run()

INFO:strategy:0.0
2015-04-20 00:00:00 strategy [INFO] 0.0
INFO:strategy:19.15
2015-04-24 00:00:00 strategy [INFO] 19.15
INFO:strategy:19.25
2015-04-27 00:00:00 strategy [INFO] 19.25
INFO:strategy:19.1
2015-04-28 00:00:00 strategy [INFO] 19.1
INFO:strategy:19.25
2015-04-29 00:00:00 strategy [INFO] 19.25
INFO:strategy:19.3
2015-04-30 00:00:00 strategy [INFO] 19.3
INFO:strategy:19.05
2015-05-01 00:00:00 strategy [INFO] 19.05
INFO:strategy:19.2
2015-05-04 00:00:00 strategy [INFO] 19.2
INFO:strategy:19.25
2015-05-05 00:00:00 strategy [INFO] 19.25
INFO:strategy:19.35
2015-05-06 00:00:00 strategy [INFO] 19.35
INFO:strategy:19.3
2015-05-07 00:00:00 strategy [INFO] 19.3
INFO:strategy:19.1
2015-05-08 00:00:00 strategy [INFO] 19.1
INFO:strategy:19.33
2015-05-11 00:00:00 strategy [INFO] 19.33
INFO:strategy:19.3
2015-05-12 00:00:00 strategy [INFO] 19.3
INFO:strategy:19.15
2015-05-13 00:00:00 strategy [INFO] 19.15
INFO:strategy:19.04
2015-05-14 00:00:00 strategy [INFO] 19.04
INFO:strategy:19.0
2015-

## TO-DO: Incorporate VIX, multiple VIX futures and calculate VRP